SDG partier via [partiguiden.se](https://partiguiden.nu/) / [GITHUB](https://github.com/Ackuq/partiguiden/issues/1249)
* [SDG mapper](https://knowsdgs.jrc.ec.europa.eu/sdgmapper)  

* API https://backend.partiguiden.se/standpoints  

This [Notebook](https://github.com/salgo60/SDGMapperSweden/blob/main/Notebook/partiguiden%20SDG.ipynb)

#### Version
0.2 supports source_language see https://knowsdgs.jrc.ec.europa.eu/sdgmapper/api-doc

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-07-28 22:00:33.814317


In [2]:
import pandas
import urllib3, json
import pandas as pd   
http = urllib3.PoolManager() 

doc_source_language = "sv"

def getdf(url):
    r = http.request('GET', url) 
    data = json.loads(r.data)
    return pd.json_normalize(data)

urlStandpoints = "https://backend.partiguiden.se/standpoints"

dfStandpoints = getdf(urlStandpoints)  
dfStandpoints

,id,title,content,date,link,party,subject
0,aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG...,Abort,[Att fria och säkra aborter ska vara en mänskl...,2022-06-25,https://www.centerpartiet.se/var-politik/polit...,C,15.0
1,aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG...,Abort,[Rätt till fria och säkra aborter till och med...,2022-06-25,https://www.centerpartiet.se/var-politik/polit...,C,2.0
2,aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG...,Adoption,[Barnets bästa ska styra när det handlar om ad...,2022-06-25,https://www.centerpartiet.se/var-politik/polit...,C,7.0
3,aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG...,A-kassa och omställningsförsäkring,[Att a-kassan ska vara allmän och omfatta alla...,2022-06-25,https://www.centerpartiet.se/var-politik/polit...,C,20.0
4,aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG...,Alkoholpolitik,[Tillåta gårdsförsäljning av alkoholhaltiga dr...,2022-06-25,https://www.centerpartiet.se/var-politik/polit...,C,2.0
...,...,...,...,...,...,...,...
1186,aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saX...,Vapen­export,[Vänsterpartiet vill att Sverige ska vara ett ...,2022-06-25,https://www.vansterpartiet.se/politik/vapenexp...,V,9.0
1187,aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saX...,Vinster i välfärden,[När vi går till vårdcentralen vill vi vara tr...,2022-06-25,https://www.vansterpartiet.se/politik/vinster-...,V,20.0
1188,aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saX...,Vålds­bejakande extremism,[Våldsbejakande extremism och terrorism är hot...,2022-06-25,https://www.vansterpartiet.se/politik/valdsbej...,V,9.0
1189,aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saX...,Välfärd,[Att välfärden fungerar är en förutsättning fö...,2022-06-25,https://www.vansterpartiet.se/politik/valfard/,V,20.0


In [3]:
import requests,time,json
urlSDGMapper = "https://knowsdgs.jrc.ec.europa.eu/"
s = requests.Session()
tkn = s.post(urlSDGMapper + "session/token")
from tqdm import tqdm 
def SDGMapperText(doc_txt):
    #time.sleep(6)
    myKey = "db0c87cbdf761defc71ea93010fdde41"

    headers = {"Content-Type": "application/json", "X-Csrf-Token": tkn.text, "X-Api-Key": myKey }
    rSDG = s.post(urlSDGMapper + "api/rest/mappingdata", headers=headers, json={
            "input_text": doc_txt,
            "source_language" : doc_source_language})
    #print(rSDG)
    jsonSDG = json.loads(rSDG.text)
    #print(jsonSDG)
    return jsonSDG 
numberSDGinStandpointsText = 0
for index,row in tqdm(dfStandpoints.iterrows(), total=dfStandpoints.shape[0]):
    try:
        standpointsText = row["title"] + " " +  " ".join(row["content"])
        SDGret = SDGMapperText(standpointsText)
        SDGjson = json.loads(SDGret["data"][0])

        if len(SDGjson['children']) > 0:
            print(row["id"],"\n",row["link"],"\n\n",standpointsText,"\n\t",SDGjson['children'],"\n")
            numberSDGinStandpointsText += 1
    except Exception as  e: 
        print("\t Error: ",index,"\n",e)
print("SDG",numberSDGinStandpointsText)

  0%|          | 2/1191 [00:09<1:39:23,  5.02s/it]

aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG9saXRpay9wb2xpdGlrLWEtby92YXJkLW9jaC1vbXNvcmcvYWJvcnQ= 
 https://www.centerpartiet.se/var-politik/politik-a-o/vard-och-omsorg/abort 

 Abort Rätt till fria och säkra aborter till och med 18:e graviditetsveckan Satsa på förebyggande arbete för att minska antalet aborter Arbeta för att aborter ska tillåtas i alla EU:s medlemsstater 
	 [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Goal/3/Target/List?includechildren=false', '@type': 'sdgo:Goal', 'name': 'SDG 3', 'n_occurrences': 1, 'relevance': '100.0%', 'children': [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Target/3.7/Indicator/List?includechildren=false', '@type': 'sdgo:Target', 'name': 'Target 3.7', 'n_occurrences': 1}]}] 



  0%|          | 4/1191 [00:20<1:38:58,  5.00s/it]

aHR0cHM6Ly93d3cuY2VudGVycGFydGlldC5zZS92YXItcG9saXRpay9wb2xpdGlrLWEtby90cnlnZ2hldHNzeXN0ZW0vYS1rYXNzYS1vY2gtb21zdGFsbG5pbmdzZm9yc2FrcmluZw== 
 https://www.centerpartiet.se/var-politik/politik-a-o/trygghetssystem/a-kassa-och-omstallningsforsakring 

 A-kassa och omställningsförsäkring Att a-kassan ska vara allmän och omfatta alla som har kvalificerat sig. Se en högre nivå på a-kassan under de första månaderna av arbetslösheten, en ersättning som sedan trappas ner. 
	 [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Goal/8/Target/List?includechildren=false', '@type': 'sdgo:Goal', 'name': 'SDG 8', 'n_occurrences': 2, 'relevance': '100.0%', 'children': [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Target/8.5/Indicator/List?includechildren=false', '@type': 'sdgo:Target', 'name': 'Target 8.5', 'n_occurrences': 2}]}] 



  1%|          | 7/1191 [00:35<1:40:25,  5.09s/it]

...

aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saXRpay9zb2NpYWxwb2xpdGlrLw== 
 https://www.vansterpartiet.se/politik/socialpolitik/ 

 Social­poli­tik Alla har rätt till ett skyddsnät som fångar upp oss när sjukdom eller arbetslöshet gör att vi faller. Det ska inte bara finnas för de som har pengar eller anhöriga som kämpar för dem. Ett jämlikt samhälle med få människor som lever i utsatthet är bra för alla. 
	 [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Goal/8/Target/List?includechildren=false', '@type': 'sdgo:Goal', 'name': 'SDG 8', 'n_occurrences': 1, 'relevance': '100.0%', 'children': [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Target/8.5/Indicator/List?includechildren=false', '@type': 'sdgo:Target', 'name': 'Target 8.5', 'n_occurrences': 1}]}] 



100%|█████████▉| 1189/1191 [1:42:54<00:10,  5.18s/it]

aHR0cHM6Ly93d3cudmFuc3RlcnBhcnRpZXQuc2UvcG9saXRpay92YWxkc2JlamFrYW5kZS1leHRyZW1pc20v 
 https://www.vansterpartiet.se/politik/valdsbejakande-extremism/ 

 Vålds­bejakande extremism Våldsbejakande extremism och terrorism är hot mot grundläggande demokratiska värden och det demokratiska samhället. Vi måste ta det på största allvar, oavsett om det sker i Syrien, Paris, Stockholm eller på nätet. 
	 [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Goal/16/Target/List?includechildren=false', '@type': 'sdgo:Goal', 'name': 'SDG 16', 'n_occurrences': 1, 'relevance': '100.0%', 'children': [{'@id': 'https://unstats.un.org/SDGAPI/v1/sdg/Target/16.1/Indicator/List?includechildren=false', '@type': 'sdgo:Target', 'name': 'Target 16.1', 'n_occurrences': 1}]}] 



100%|██████████| 1191/1191 [1:43:05<00:00,  5.19s/it]

SDG 557


## Resultat
### version 0.1 
* 154 av 1191 dokument gjordes någon koppling till SDG
* Target 9.5 var vanligaste 
* jag har fått feedback från utvecklaren om att översätta till engelska innan
  * tycker dock att ett EU API borde kunna svenska
  
### version 0.2
* stödjer, nu svenska
* **557 av 1191 dokument** gjordes någon koppling SDG

In [4]:
print("End run: ", datetime.now())

End run:  2022-07-28 23:43:40.670620
